# Load all the data from the input files
1. Training Dataset : Merge the quantitative, categorical metadata files & solution data
2. Training Dataset : Merge the quantitative, categorical metadata files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import os

# Set base directory for data
BASE_DIR = r"C:\Users\Maab\Desktop\ADHD_Kaggle_Competition\Repo\WiDS-Datathon-2025\Data\raw"

# Function to load all data
def get_feats(mode='TRAIN'):
    """
    Loads and merges the relevant datasets based on the mode ('TRAIN' or 'TEST').
    """

    # Define folder path based on mode
    folder = os.path.join(BASE_DIR, "TRAIN_NEW" if mode == 'TRAIN' else "TEST")

    # Load quantitative metadata
    feats = pd.read_excel(os.path.join(folder, f"{mode}_QUANTITATIVE_METADATA_new.xlsx" if mode == 'TRAIN' else f"{mode}_QUANTITATIVE_METADATA.xlsx"))

    # Load categorical metadata
    if mode == 'TRAIN':
        cate = pd.read_excel(os.path.join(folder, "TRAIN_CATEGORICAL_METADATA_new.xlsx"))
    else:
        cate = pd.read_excel(os.path.join(folder, "TEST_CATEGORICAL.xlsx"))

    # Merge categorical data
    feats = feats.merge(cate, on='participant_id', how='left')

    # Load functional connectome matrices
    func_filename = "TRAIN_FUNCTIONAL_CONNECTOME_MATRICES_new_36P_Pearson.csv" if mode == 'TRAIN' else "TEST_FUNCTIONAL_CONNECTOME_MATRICES.csv"
    func = pd.read_csv(os.path.join(folder, func_filename))

    # Merge functional data
    feats = feats.merge(func, on='participant_id', how='left')

    # If training data, merge with solution file
    if mode == 'TRAIN':
        solution_path = os.path.join(folder, "TRAINING_SOLUTIONS.xlsx")
        solution = pd.read_excel(solution_path)
        feats = feats.merge(solution, on='participant_id', how='left')

    return feats


In [3]:
#Load data
train = get_feats(mode='TRAIN')
test = get_feats(mode='TEST')

sub = pd.read_excel(os.path.join(BASE_DIR, 'SAMPLE_SUBMISSION.xlsx'))
y = pd.read_excel(os.path.join(BASE_DIR, "TRAIN_NEW\TRAINING_SOLUTIONS.xlsx"))


# Set index
train.set_index('participant_id', inplace=True)
test.set_index('participant_id', inplace=True)

# Define targets and features
targets = ['ADHD_Outcome', 'Sex_F']

non_connectome_features = [
    feature for feature in train.columns 
    if 'throw' not in feature and feature not in targets
]

connectome_features = [feature for feature in train.columns if 'throw' in feature]

# Dropping Features

In [4]:
##removing columns from train that is not in test
y_ADHD = train['ADHD_Outcome']
y_Sex = train['Sex_F']

train.drop(columns=['ADHD_Outcome', 'Sex_F'], inplace=True)

In [5]:
drop_cols = [
    "Basic_Demos_Study_Site", "MRI_Track_Scan_Location", 'Barratt_Barratt_P1_Occ', 'Barratt_Barratt_P2_Occ'
]

train.drop(drop_cols, axis=1, inplace=True)
test.drop(drop_cols, axis=1, inplace=True)

# MISSING VALUES

### Impute train and test set for missing values

In [6]:
from sklearn.experimental import enable_iterative_imputer  # Enables IterativeImputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LassoCV
import pandas as pd
import warnings
from sklearn.exceptions import ConvergenceWarning

# Suppress the convergence warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)


# Check for missing values
missing = train.isnull().sum()
missing_percent = 100 * missing / len(train)
missing_df = pd.DataFrame({'Missing Values': missing, 'Percentage': missing_percent})

# Select columns to impute (first 23 columns, for example)
cols_to_impute = train.columns[:23]

# Initialize the imputer
imputer = IterativeImputer(estimator=LassoCV(random_state=42), max_iter=5, random_state=42)

# Impute in-place
train[cols_to_impute] = imputer.fit_transform(train[cols_to_impute])
test[cols_to_impute] = imputer.transform(test[cols_to_impute])

# Print features that originally had missing values
missing_features = missing_df[missing_df['Missing Values'] > 0].sort_values('Percentage', ascending=False)
print("\nFeatures with missing values in training data before imputing:")
print(missing_features)



Features with missing values in training data before imputing:
                                  Missing Values  Percentage
MRI_Track_Age_at_Scan                        360   29.678483
Barratt_Barratt_P2_Edu                       198   16.323166
PreInt_Demos_Fam_Child_Race                   54    4.451772
PreInt_Demos_Fam_Child_Ethnicity              43    3.544930
ColorVision_CV_Score                          23    1.896125
Barratt_Barratt_P1_Edu                        15    1.236603
EHQ_EHQ_Total                                 13    1.071723
APQ_P_APQ_P_PP                                12    0.989283
APQ_P_APQ_P_PM                                12    0.989283
APQ_P_APQ_P_OPD                               12    0.989283
APQ_P_APQ_P_INV                               12    0.989283
APQ_P_APQ_P_ID                                12    0.989283
APQ_P_APQ_P_CP                                12    0.989283
SDQ_SDQ_Conduct_Problems                       9    0.741962
SDQ_SDQ_Difficulties_

In [7]:
# Check for missing values in the testing data
missing = train.isnull().sum()
missing_percent = 100 * missing / len(train)
missing_df = pd.DataFrame({
    'Missing Values': missing,
    'Percentage': missing_percent
})


# Display features with missing values
missing_features = missing_df[missing_df['Missing Values'] > 0].sort_values('Percentage', ascending=False)
print("\nFeatures with missing values in testing data:")
missing_features



Features with missing values in testing data:


,Missing Values,Percentage


# Standarization

In [8]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
# Standardize features
scaler = StandardScaler()
train = pd.DataFrame(
    scaler.fit_transform(train), columns=train.columns, index=train.index
)
test = pd.DataFrame(
    scaler.transform(test), columns=test.columns, index=test.index
)

# MODELING

In [9]:

features_sex = [
    'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP',
    'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
    'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
    'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems',
    'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact',
    'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing',
    'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan'
]


features_adhd = [
    'EHQ_EHQ_Total', 'ColorVision_CV_Score', 'APQ_P_APQ_P_CP',
    'APQ_P_APQ_P_ID', 'APQ_P_APQ_P_INV', 'APQ_P_APQ_P_OPD',
    'APQ_P_APQ_P_PM', 'APQ_P_APQ_P_PP', 'SDQ_SDQ_Conduct_Problems',
    'SDQ_SDQ_Difficulties_Total', 'SDQ_SDQ_Emotional_Problems',
    'SDQ_SDQ_Externalizing', 'SDQ_SDQ_Generating_Impact',
    'SDQ_SDQ_Hyperactivity', 'SDQ_SDQ_Internalizing',
    'SDQ_SDQ_Peer_Problems', 'SDQ_SDQ_Prosocial', 'MRI_Track_Age_at_Scan', 
    'PreInt_Demos_Fam_Child_Race'
]


# Features to be interacted with predicted probability of Sex_F = 1
interactions = [
    "APQ_P_APQ_P_INV", "APQ_P_APQ_P_PP", "SDQ_SDQ_Hyperactivity", 
    "MRI_Track_Age_at_Scan", "SDQ_SDQ_Generating_Impact"
]

combinations = y_ADHD.astype(str) + y_Sex.astype(str)


1. L1 gives better F1 score for ADHD, but ROC AUC is veeerrrryyyyyy low
2. L2 gives better score for sex (both F1 and ROC AUC)

## Penalty L2 Both Basic Logistic Regression


### 🧪 Repeated Stratified Cross-Validation with Logistic RegressionCV: Predicting Sex and ADHD
This section performs **repeated stratified K-Fold cross-validation** for two related classification tasks using `LogisticRegressionCV`: predicting **sex (Sex_F)** and **ADHD outcome**. It incorporates interaction terms and sample weighting to improve prediction robustness.

---

### 🔧 Models:
- **Model**: `LogisticRegressionCV` with L2 penalty, saga solver, inner StratifiedKFold CV.
- **Task 1 (Sex Prediction)**:
  - **Target**: `y_Sex`
  - **Features**: Full feature set from `train`
- **Task 2 (ADHD Prediction)**:
  - **Target**: `y_ADHD`
  - **Features**: `features_adhd` + `sex_proba` + interactions between `sex_proba` and selected features

---

### 🎯 Key Features:
- Uses **predicted sex probabilities** as a feature for ADHD prediction.
- **Interaction terms**: Multiplicative features between `sex_proba` and selected predictors.
- Sample weights:
  - Doubles the weight for samples where both labels (`Sex_F` and `ADHD`) are positive (`"11"`) to handle data imbalance.

---

### 🔄 Validation Strategy:
- **RepeatedStratifiedKFold** (5 folds × 5 repeats)
- Performance metrics:
  - **F1 Score** (thresholded)
  - **ROC-AUC**
- Out-of-fold predictions are collected for final performance reporting.

---

### 🧮 Outputs:
- Final mean F1 and ROC-AUC for both `Sex_F` and `ADHD` predictions.

### 🧠 Insights from Cross-Validation

### 1. **Sex Prediction (Sex_F) Performance**
   - **Mean F1**: 0.7822
   - **Mean ROC-AUC**: 0.7872
   - **Key Observations**:
     - The **F1 scores** for sex prediction vary, with the highest reaching 0.7268 in Fold 22 and the lowest at 0.6534 in Fold 15. This suggests fluctuation in model performance across data splits.
     - The **ROC-AUC** remains stable, indicating the model consistently distinguishes between classes with good confidence.
     - **Fluctuations in performance** (especially in folds like 15 and 17) suggest the model might struggle with certain data subsets, indicating room for improvement.

### 2. **ADHD Outcome Prediction (Outcome ADHD) Performance**
   - **Mean F1**: 0.8370
   - **Mean ROC-AUC**: 0.8427
   - **Key Observations**:
     - **F1 scores** for ADHD prediction are consistently strong, with scores above 0.85 in most folds. The highest F1 score (Fold 24) reaches 0.8968, showcasing strong model performance.
     - **ROC-AUC** is consistently high, showing the model's strong ability to differentiate between ADHD and non-ADHD cases.

### 3. **Comparison of Tasks (Sex vs. ADHD)**
   - **ADHD prediction** consistently outperforms **Sex_F** in both **F1** and **ROC-AUC** scores. This could be due to:
     - Better separation or more predictable patterns in ADHD data.
     - Potentially more informative features for ADHD prediction.

### 4. **Model Stability and Performance Fluctuations**
   - The **model's performance** fluctuates across folds for both tasks, but **ADHD** consistently outperforms **Sex_F**. 
   - The **fluctuations** in **Sex_F performance**, particularly in folds 15 and 17, suggest that the model may be struggling with certain data subsets, especially in sex prediction where there is more variance.

---

### 🎯 Final Takeaways:
- Both models show **promising results**, with **ADHD prediction** being the stronger task overall.
- The **Sex_F performance** shows **room for improvement**, especially with the fluctuations in some folds. Consider feature engineering, tuning, or exploring other models to enhance performance.
- The **ADHD model** is robust and reliable across folds, demonstrating **solid predictive power** for this task.


In [ ]:
#Actual Validation and Modeling
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score, f1_score
import numpy as np

# Constants
SEED = 42
REPEATS = 5
FOLDS = 5
t_sex = 0.3
t_adhd = 0.4

# Evaluation function


def eval_metrics(y_true, y_pred, weights, label="None", thresh=0.5):
    """Evaluate predictions using F1 Score and ROC-AUC."""
    
    # Calculate the ROC-AUC score (works with probabilities for multi-class classification too)
    roc_auc = roc_auc_score(y_true, y_pred)
    
    # Calculate the F1 score with thresholding, using weights for sample weighting
    f1 = f1_score(y_true, (y_pred > thresh).astype(int), sample_weight=weights)
    
    # Print the results
    print(f"{label} -> F1: {f1:.4f}, ROC-AUC: {roc_auc:.4f}")
    
    # Return the metrics
    return f1, roc_auc


# Initialize lists to store scores and out-of-fold predictions
scores_sex = []
scores_adhd = []
sex_oof = np.zeros(len(y_Sex))
adhd_oof = np.zeros(len(y_ADHD))

# Cross-validation setup
rskf = RepeatedStratifiedKFold(n_splits=FOLDS, n_repeats=REPEATS, random_state=SEED)
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

# Logistic RegressionCV parameters
logreg_params = {
    "penalty": "l2",
    "Cs": 10,
    "cv": skf,
    "fit_intercept": True,
    "scoring": "f1",
    "random_state": SEED,
    "solver": "saga"
    # "max_iter": 1000  # Increase this

}

model_sex = LogisticRegressionCV(**logreg_params)
model_adhd = LogisticRegressionCV(**logreg_params)

# Start cross-validation loop
for fold, (train_idx, val_idx) in enumerate(rskf.split(train, combinations), 1):
    print(f"\n=== Fold {fold} ===")

    # Split data
    X_train, X_val = train.iloc[train_idx].copy(), train.iloc[val_idx].copy()
    y_train_sex, y_val_sex = y_Sex.iloc[train_idx], y_Sex.iloc[val_idx]
    y_train_adhd, y_val_adhd = y_ADHD.iloc[train_idx], y_ADHD.iloc[val_idx]

    # Sample weights
    weights_train = np.where(combinations.iloc[train_idx] == "11", 2, 1)
    weights_val = np.where(combinations.iloc[val_idx] == "11", 2, 1)

    # Train model to predict Sex_F
    model_sex.fit(X_train, y_train_sex, sample_weight=weights_train)
    sex_train_pred = model_sex.predict_proba(X_train)[:, 1]
    sex_val_pred = model_sex.predict_proba(X_val)[:, 1]
    sex_oof[val_idx] += sex_val_pred / REPEATS

    sex_f1, sex_roc_auc = eval_metrics(y_val_sex, sex_val_pred, weights_val, "Sex_F", thresh=t_sex)
    scores_sex.append((sex_f1, sex_roc_auc))

    # Add predicted sex probabilities
    X_train = X_train.assign(sex_proba=sex_train_pred)
    X_val = X_val.assign(sex_proba=sex_val_pred)

    # Create interaction features
    for col in interactions:
        X_train[f"I_{col}"] = X_train[col] * X_train["sex_proba"]
        X_val[f"I_{col}"] = X_val[col] * X_val["sex_proba"]

    # Train model to predict ADHD outcome
    model_adhd.fit(X_train[features_adhd], y_train_adhd, sample_weight=weights_train)
    adhd_val_pred = model_adhd.predict_proba(X_val[features_adhd])[:, 1]
    adhd_oof[val_idx] += adhd_val_pred / REPEATS

    adhd_f1, adhd_roc_auc = eval_metrics(y_val_adhd, adhd_val_pred, weights_val, "Outcome ADHD", thresh=t_adhd)
    scores_adhd.append((adhd_f1, adhd_roc_auc))

# Print final results
print(f"\n=== CV Results ===")
print(f"Sex Mean roc auc Score: {np.mean([b for b, _ in scores_sex]):.4f}")
print(f"Sex Mean F1: {np.mean([f for _, f in scores_sex]):.4f}")
print(f"ADHD Mean roc auc Score: {np.mean([b for b, _ in scores_adhd]):.4f}")
print(f"ADHD Mean F1: {np.mean([f for _, f in scores_adhd]):.4f}")



=== Fold 1 ===
Sex_F -> F1: 0.6967, ROC-AUC: 0.7680
Outcome ADHD -> F1: 0.8839, ROC-AUC: 0.8603

=== Fold 2 ===
Sex_F -> F1: 0.7086, ROC-AUC: 0.8125
Outcome ADHD -> F1: 0.8812, ROC-AUC: 0.8291

=== Fold 3 ===
Sex_F -> F1: 0.7082, ROC-AUC: 0.7977
Outcome ADHD -> F1: 0.8860, ROC-AUC: 0.8309

=== Fold 4 ===
Sex_F -> F1: 0.7163, ROC-AUC: 0.7910
Outcome ADHD -> F1: 0.8742, ROC-AUC: 0.8572

=== Fold 5 ===
Sex_F -> F1: 0.6817, ROC-AUC: 0.8046
Outcome ADHD -> F1: 0.8945, ROC-AUC: 0.7982

=== Fold 6 ===
Sex_F -> F1: 0.6908, ROC-AUC: 0.7813
Outcome ADHD -> F1: 0.8645, ROC-AUC: 0.8435

=== Fold 7 ===
Sex_F -> F1: 0.7035, ROC-AUC: 0.7385
Outcome ADHD -> F1: 0.8683, ROC-AUC: 0.8212

=== Fold 8 ===
Sex_F -> F1: 0.6799, ROC-AUC: 0.7466
Outcome ADHD -> F1: 0.8747, ROC-AUC: 0.8135

=== Fold 9 ===
Sex_F -> F1: 0.7082, ROC-AUC: 0.8193
Outcome ADHD -> F1: 0.8855, ROC-AUC: 0.8496

=== Fold 10 ===
Sex_F -> F1: 0.6782, ROC-AUC: 0.7733
Outcome ADHD -> F1: 0.9204, ROC-AUC: 0.8692

=== Fold 11 ===
Sex_F -> F1:

- Kaggle Description: 
Logistic Regression models used for predicting both 'Sex_F' and 'Outcome_ADHD' using Repeated Stratified K-Folds (5-fold, 5 repeats). Key tests included:
1. Feature Engineering: Added interaction features using predicted sex probabilities, which improved ADHD prediction.
2. Sex Prediction: The model showed fluctuations in performance, with F1 scores ranging from 0.6534 to 0.7268, but stable ROC-AUC.
3. ADHD Prediction: Strong and consistent performance, with F1 scores mostly above 0.85 and high ROC-AUC.


Logistic Regression models used for predicting both 'Sex_F' and 'Outcome_ADHD' using Repeated Stratified K-Folds (5-fold, 5 repeats). Key tests included:
1. model outputs probabilities, initially in my last submission I made the threshold to consider sex to be female 0.4 (ADHD was 0.5), in this submission I made it 0.5 for sex
1. Feature Engineering: Added interaction features using predicted sex probabilities, which improved ADHD prediction.
2. Sex Prediction: The model showed fluctuations in performance, with F1 scores ranging from 0.6534 to 0.7268, but stable ROC-AUC.
3. ADHD Prediction: Strong and consistent performance, with F1 scores mostly above 0.85 and high 

## Submission

In [ ]:
    # ==========================================
    # ==========================================
    # ==========================================

    # Resulting file isn't in 1 or 0, it's in probabilities 

    # ==========================================
    # ==========================================
    # ==========================================



# Logistic RegressionCV parameters
logreg_params = {
    "penalty": "l2",
    "Cs": 10,
    "cv": skf,
    "fit_intercept": True,
    "scoring": "f1",
    "random_state": SEED,
    "solver": "saga"
    # "max_iter": 1000  # Increase this

}

model_sex = LogisticRegressionCV(**logreg_params)
model_adhd = LogisticRegressionCV(**logreg_params)

# Train the models on the full dataset (without cross-validation)
model_sex.fit(train, y_Sex, sample_weight=np.where(combinations == "11", 2, 1))
model_adhd.fit(train[features_adhd], y_ADHD, sample_weight=np.where(combinations == "11", 2, 1))

# Predict on the test data
sex_test_pred = model_sex.predict_proba(test)[:, 1]  # Sex_F predictions
adhd_test_pred = model_adhd.predict_proba(test[features_adhd])[:, 1]  # ADHD predictions

# Create submission DataFrame (adjust column names as needed)
submission = pd.DataFrame({
    'participant_id': test.index,  
    'adhd_prediction': adhd_test_pred,
    'sex_prediction': sex_test_pred
})

# Save submission file
submission.to_csv('sex7822adhd8370.csv', index=False)

print("Submission file sex7822adhd8370.csv created successfully!")

# Kaggle Description: 
# Logistic Regression models used for predicting both 'Sex_F' and 'Outcome_ADHD' using Repeated Stratified K-Folds (5-fold, 5 repeats). Key tests included:
# 1. Feature Engineering: Added interaction features using predicted sex probabilities, which improved ADHD prediction.
# 2. Sex Prediction: The model showed fluctuations in performance, with F1 scores ranging from 0.6534 to 0.7268, but stable ROC-AUC.
# 3. ADHD Prediction: Strong and consistent performance, with F1 scores mostly above 0.85 and high ROC-AUC.

Submission file sex7822adhd8370.csv created successfully!


In [ ]:
    # ==========================================
    # ==========================================
    # ==========================================

    # Convert results from probabilities to actual 1s or 0s

    # ==========================================
    # ==========================================
    # ==========================================



# Convert probabilities to binary predictions (0 or 1)
sex_test_pred_binary = (sex_test_pred >= 0.3).astype(int)
adhd_test_pred_binary = (adhd_test_pred >= 0.4).astype(int)

# Create submission DataFrame with hard labels
submission = pd.DataFrame({
    'participant_id': test.index,  
    'adhd_prediction': adhd_test_pred_binary,
    'sex_prediction': sex_test_pred_binary
})

# Save submission file
submission.to_csv('Sex7822ADHD8370_ADHDThreshold4SexThreshold3.csv', index=False)
print("Binary-label submission file created successfully!")


NameError: name 'sex_test_pred' is not defined

Thre

In [20]:
import pandas as pd

# Load the CSV file with probabilities
df = pd.read_csv('sex7822adhd8370.csv')

# Apply thresholds to convert probabilities to binary labels
df['adhd_prediction'] = (df['adhd_prediction'] >= 0.4).astype(int)
df['sex_prediction'] = (df['sex_prediction'] >= 0.3).astype(int)

# Save the new binary-label submission
df.to_csv('Sex7822ADHD8370_ADHDThreshold4SexThreshold3.csv', index=False)

print("Binary-label submission file created successfully!")


Binary-label submission file created successfully!
